In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
import os, joblib

print("\n==================== CLEAN + PREPROCESS PIPELINE START ====================\n")

# ---------------------------------------------------------
# 1. LOAD RAW DATA
# ---------------------------------------------------------
file_path = "/Users/praveenkumardevamane/Downloads/Dynamic pricing/DynamicPricing2/data/retail_ecommerce_dataset_10k.csv"
data = pd.read_csv(file_path)

print("Raw data loaded. Shape:", data.shape)

In [ ]:
# 2. BASIC CLEANING
# ---------------------------------------------------------
# Convert numeric safely
for col in ["Current Price", "Competitor Price", "Historical Price",
            "Inventory Level", "Purchase Quantity", "Total Revenue",
            "Competitor Popularity", "Price Elasticity"]:
    if col in data.columns:
        data[col] = pd.to_numeric(data[col], errors="coerce")

# Convert timestamp
data["Transaction Timestamp"] = pd.to_datetime(data["Transaction Timestamp"], errors="coerce")

# Fill numeric NaN with median
num_cols_basic = data.select_dtypes(include=["number"]).columns
for col in num_cols_basic:
    if data[col].isnull().any():
        data[col] = data[col].fillna(data[col].median())

# Drop rows where timestamp missing
before = len(data)
data = data.dropna(subset=["Transaction Timestamp"])
print(f"Dropped {before - len(data)} rows with missing timestamp.")

In [ ]:
# ---------------------------------------------------------
# 3. STANDARDIZE CATEGORY TEXT
# ---------------------------------------------------------
def clean_text(x):
    if pd.isna(x): return "Unknown"
    return str(x).strip().replace(" ", "_")

for col in ["Product Name", "Product Category", "Customer Demographics", "Customer Region"]:
    if col in data.columns:
        data[col] = data[col].apply(clean_text)


In [ ]:
# ---------------------------------------------------------
# 4. SORT FOR CORRECT ROLLINGS
# ---------------------------------------------------------
data = data.sort_values(by=["Product ID", "Transaction Timestamp"])

In [ ]:
# ---------------------------------------------------------
# 5. FEATURE ENGINEERING (NO LAGS)
# ---------------------------------------------------------
# Rolling price features
data["rolling_avg_7_day"] = data.groupby("Product ID")["Current Price"].transform(lambda x: x.rolling(7).mean())
data["rolling_avg_30_day"] = data.groupby("Product ID")["Current Price"].transform(lambda x: x.rolling(30).mean())
data["rolling_volatility_7_day"] = data.groupby("Product ID")["Current Price"].transform(lambda x: x.rolling(7).std())

# Competitor price features
data["competitor_price_difference"] = data["Current Price"] - data["Competitor Price"]
data["competitor_price_ratio"] = data["Current Price"] / data["Competitor Price"]

# Discount % handling
data["discount_percentage"] = (
    data["Discount Applied"].astype(str).str.replace("%", "", regex=False)
).astype(float) / 100

# Price momentum (NO lag)
data["price_change_rate"] = data.groupby("Product ID")["Current Price"].pct_change().replace([np.inf, -np.inf], 0)

# Fill engineered NaNs with median
eng_cols = [
    "rolling_avg_7_day", "rolling_avg_30_day", "rolling_volatility_7_day",
    "competitor_price_difference", "competitor_price_ratio",
    "discount_percentage", "price_change_rate"
]

for col in eng_cols:
    if col in data.columns:
        data[col] = data[col].fillna(data[col].median())

print("Feature engineering completed.")



In [ ]:
# ---------------------------------------------------------
# 6. REMOVE LEAKAGE FEATURES
# ---------------------------------------------------------
leakage_cols = ["Historical Price", "Total Revenue"]
data = data.drop(columns=[c for c in leakage_cols if c in data.columns], errors="ignore")
print("Removed leakage columns:", leakage_cols)


In [ ]:
# ---------------------------------------------------------
# 7. DEFINE TARGET AND FEATURES
# ---------------------------------------------------------
y = data["Current Price"]
X = data.drop(columns=[
    "Transaction ID", "Transaction Timestamp", "Current Price",
    "Discount Applied", "Product ID"
], errors="ignore")

print("Final feature set shape:", X.shape)

In [ ]:
# ---------------------------------------------------------
# 8. SPLIT TRAIN / VAL / TEST
# ---------------------------------------------------------
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42)

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

In [ ]:
# ---------------------------------------------------------
# 9. IDENTIFY TYPES
# ---------------------------------------------------------
num_cols = X_train.select_dtypes(include=["number"]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object"]).columns.tolist()

print("\nNumeric cols:", len(num_cols))
print("Categorical cols:", len(cat_cols))


In [ ]:
from sklearn.impute import SimpleImputer

print("\nRunning SAFE imputers...")

# ============= NUMERIC IMPUTATION =============
num_cols_existing = [c for c in num_cols if c in X_train.columns]

num_imputer = SimpleImputer(strategy="median")
train_num = num_imputer.fit_transform(X_train[num_cols_existing])
val_num   = num_imputer.transform(X_val[num_cols_existing])
test_num  = num_imputer.transform(X_test[num_cols_existing])

# Convert back to DataFrame to avoid mismatches
train_num_df = pd.DataFrame(train_num, columns=num_cols_existing, index=X_train.index)
val_num_df   = pd.DataFrame(val_num,   columns=num_cols_existing, index=X_val.index)
test_num_df  = pd.DataFrame(test_num,  columns=num_cols_existing, index=X_test.index)

# Assign safely
for col in num_cols_existing:
    X_train[col] = train_num_df[col]
    X_val[col]   = val_num_df[col]
    X_test[col]  = test_num_df[col]


# ============= CATEGORICAL IMPUTATION =============
cat_cols_existing = [c for c in cat_cols if c in X_train.columns]

cat_imputer = SimpleImputer(strategy="most_frequent")
train_cat = cat_imputer.fit_transform(X_train[cat_cols_existing])
val_cat   = cat_imputer.transform(X_val[cat_cols_existing])
test_cat  = cat_imputer.transform(X_test[cat_cols_existing])

train_cat_df = pd.DataFrame(train_cat, columns=cat_cols_existing, index=X_train.index)
val_cat_df   = pd.DataFrame(val_cat,   columns=cat_cols_existing, index=X_val.index)
test_cat_df  = pd.DataFrame(test_cat,  columns=cat_cols_existing, index=X_test.index)

for col in cat_cols_existing:
    X_train[col] = train_cat_df[col]
    X_val[col]   = val_cat_df[col]
    X_test[col]  = test_cat_df[col]

print("SAFE imputation completed.")

In [ ]:
print("\nFixing numerical columns for scaling...")

for col in numerical_cols:
    # Fill NaN values with median
    data[col] = data[col].fillna(data[col].median())

    # If column has zero variance (std = 0), add tiny noise so scaler works
    if data[col].std() == 0:
        data[col] += 1e-9
        print(f"   Fixed zero-variance column: {col}")

print("Numerical columns ready for scaling.")

In [ ]:
# ---------------------------------------------------------
# 9. SCALE NUMERICAL FEATURES
# ---------------------------------------------------------
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train[numerical_cols]), columns=numerical_cols, index=X_train.index)
X_val_scaled   = pd.DataFrame(scaler.transform(X_val[numerical_cols]), columns=numerical_cols, index=X_val.index)
X_test_scaled  = pd.DataFrame(scaler.transform(X_test[numerical_cols]), columns=numerical_cols, index=X_test.index)



In [ ]:
# ---------------------------------------------------------
# 10. ENCODE CATEGORICAL FEATURES
# ---------------------------------------------------------
encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)

X_train_encoded = pd.DataFrame(encoder.fit_transform(X_train[categorical_cols]), index=X_train.index)
X_val_encoded   = pd.DataFrame(encoder.transform(X_val[categorical_cols]), index=X_val.index)
X_test_encoded  = pd.DataFrame(encoder.transform(X_test[categorical_cols]), index=X_test.index)

# Add column names
X_train_encoded.columns = encoder.get_feature_names_out(categorical_cols)
X_val_encoded.columns   = encoder.get_feature_names_out(categorical_cols)
X_test_encoded.columns  = encoder.get_feature_names_out(categorical_cols)


In [ ]:

# ---------------------------------------------------------
# 11. MERGE NUMERIC + ENCODED CATEGORICAL
# ---------------------------------------------------------
X_train_processed = pd.concat([X_train_scaled, X_train_encoded], axis=1)
X_val_processed   = pd.concat([X_val_scaled, X_val_encoded], axis=1)
X_test_processed  = pd.concat([X_test_scaled, X_test_encoded], axis=1)

print("Final processed train shape:", X_train_processed.shape)


In [ ]:
X_train_processed.isnull().sum()

In [ ]:
# ---------------------------------------------------------
# 12. SAVE PROCESSED DATA + OBJECTS
# ---------------------------------------------------------
os.makedirs("preprocessed_data", exist_ok=True)

X_train_processed.to_csv("preprocessed_data/X_train_processed.csv", index=False)
X_val_processed.to_csv("preprocessed_data/X_val_processed.csv", index=False)
X_test_processed.to_csv("preprocessed_data/X_test_processed.csv", index=False)

y_train.to_csv("preprocessed_data/y_train.csv", index=False)
y_val.to_csv("preprocessed_data/y_val.csv", index=False)
y_test.to_csv("preprocessed_data/y_test.csv", index=False)

joblib.dump(scaler, "preprocessed_data/scaler.pkl")
joblib.dump(encoder, "preprocessed_data/encoder.pkl")

with open("preprocessed_data/numerical_cols.txt", "w") as f:
    f.write("\n".join(numerical_cols))

with open("preprocessed_data/categorical_cols.txt", "w") as f:
    f.write("\n".join(categorical_cols))

print("\n==================== PREPROCESSING PIPELINE COMPLETE ====================\n")

In [1]:
# ========================= PREPROCESSING PIPELINE (NO ROLLING FEATURES) =========================

import pandas as pd
import numpy as np
import os, joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

print("\n==================== PREPROCESSING PIPELINE START ====================\n")

# -------------------------------------------------------------------------
# 1. LOAD RAW DATA
# -------------------------------------------------------------------------
file_path = "/Users/praveenkumardevamane/Downloads/Dynamic pricing/DynamicPricing2/data/retail_ecommerce_dataset_10k.csv"
data = pd.read_csv(file_path)
print("Loaded raw data:", data.shape)

# -------------------------------------------------------------------------
# 2. BASIC TYPE FIXING
# -------------------------------------------------------------------------
data["Current Price"] = pd.to_numeric(data["Current Price"], errors="coerce")
data["Competitor Price"] = pd.to_numeric(data["Competitor Price"], errors="coerce")
data["Transaction Timestamp"] = pd.to_datetime(data["Transaction Timestamp"], errors="coerce")

# Fill minimal NaNs
for col in ["Current Price", "Competitor Price"]:
    if data[col].isnull().any():
        data[col] = data[col].fillna(data[col].median())

# Drop rows without timestamps
data = data.dropna(subset=["Transaction Timestamp"])
print("After timestamp cleaning:", data.shape)

# -------------------------------------------------------------------------
# 3. SORT BY PRODUCT + TIME (Required for consistency)
# -------------------------------------------------------------------------
data = data.sort_values(by=["Product ID", "Transaction Timestamp"])


# -------------------------------------------------------------------------
# 4. FEATURE ENGINEERING (NO ROLLING FEATURES)
# -------------------------------------------------------------------------

# Competitor features
data["competitor_price_difference"] = data["Current Price"] - data["Competitor Price"]
data["competitor_price_ratio"] = data["Current Price"] / data["Competitor Price"]

# Discount %
data["discount_percentage"] = (
    data["Discount Applied"]
    .astype(str)
    .str.replace("%", "", regex=False)
    .astype(float)
    / 100
).fillna(0)

# Price elasticity proxy — safe & simple
data["price_change_rate"] = (
    data.groupby("Product ID")["Current Price"].diff().fillna(0)
)

print("Feature engineering complete (NO rolling features).")

# -------------------------------------------------------------------------
# 5. REMOVE LEAKAGE FEATURES
# -------------------------------------------------------------------------
leakage_cols = ["Historical Price", "Total Revenue"]

for col in leakage_cols:
    if col in data.columns:
        data = data.drop(columns=col)

print("Removed leakage features:", leakage_cols)

# -------------------------------------------------------------------------
# 6. DEFINE X AND y
# -------------------------------------------------------------------------
y = data["Current Price"]

drop_cols = ["Transaction ID", "Transaction Timestamp", "Current Price", "Discount Applied", "Product ID"]
X = data.drop(columns=[c for c in drop_cols if c in data.columns])

print("Feature matrix shape:", X.shape)

# -------------------------------------------------------------------------
# 7. TRAIN / VAL / TEST SPLIT
# -------------------------------------------------------------------------
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42)

print("Train:", X_train.shape, "| Val:", X_val.shape, "| Test:", X_test.shape)

# -------------------------------------------------------------------------
# 8. IDENTIFY COLUMN TYPES
# -------------------------------------------------------------------------
num_cols = X.select_dtypes(include=["number"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object"]).columns.tolist()

print("Numeric columns:", num_cols)
print("Categorical columns:", cat_cols)

# -------------------------------------------------------------------------
# 9. IMPUTE NUMERIC COLUMNS SAFELY
# -------------------------------------------------------------------------
num_imputer = SimpleImputer(strategy="median")

X_train[num_cols] = num_imputer.fit_transform(X_train[num_cols])
X_val[num_cols]   = num_imputer.transform(X_val[num_cols])
X_test[num_cols]  = num_imputer.transform(X_test[num_cols])

print("Numeric imputation complete.")

# -------------------------------------------------------------------------
# 10. IMPUTE CATEGORICAL COLUMNS SAFELY
# -------------------------------------------------------------------------
cat_imputer = SimpleImputer(strategy="most_frequent")

X_train[cat_cols] = cat_imputer.fit_transform(X_train[cat_cols])
X_val[cat_cols]   = cat_imputer.transform(X_val[cat_cols])
X_test[cat_cols]  = cat_imputer.transform(X_test[cat_cols])

print("Categorical imputation complete.")

# -------------------------------------------------------------------------
# 11. SCALE NUMERIC FEATURES
# -------------------------------------------------------------------------
scaler = StandardScaler()

X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train[num_cols]), columns=num_cols, index=X_train.index)
X_val_scaled   = pd.DataFrame(scaler.transform(X_val[num_cols]), columns=num_cols, index=X_val.index)
X_test_scaled  = pd.DataFrame(scaler.transform(X_test[num_cols]), columns=num_cols, index=X_test.index)

# -------------------------------------------------------------------------
# 12. ONE-HOT ENCODE CATEGORICAL FEATURES
# -------------------------------------------------------------------------
encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)

train_cat = encoder.fit_transform(X_train[cat_cols])
val_cat   = encoder.transform(X_val[cat_cols])
test_cat  = encoder.transform(X_test[cat_cols])

train_cat_df = pd.DataFrame(train_cat, columns=encoder.get_feature_names_out(cat_cols), index=X_train.index)
val_cat_df   = pd.DataFrame(val_cat,   columns=encoder.get_feature_names_out(cat_cols), index=X_val.index)
test_cat_df  = pd.DataFrame(test_cat,  columns=encoder.get_feature_names_out(cat_cols), index=X_test.index)

# -------------------------------------------------------------------------
# 13. MERGE FEATURES
# -------------------------------------------------------------------------
X_train_processed = pd.concat([X_train_scaled, train_cat_df], axis=1)
X_val_processed   = pd.concat([X_val_scaled,   val_cat_df],   axis=1)
X_test_processed  = pd.concat([X_test_scaled,  test_cat_df],  axis=1)

print("Processed shapes:",
      X_train_processed.shape,
      X_val_processed.shape,
      X_test_processed.shape)

# -------------------------------------------------------------------------
# 14. SAVE EVERYTHING
# -------------------------------------------------------------------------
os.makedirs("preprocessed_data", exist_ok=True)

X_train_processed.to_csv("preprocessed_data/X_train_processed.csv", index=False)
X_val_processed.to_csv("preprocessed_data/X_val_processed.csv", index=False)
X_test_processed.to_csv("preprocessed_data/X_test_processed.csv", index=False)

y_train.to_csv("preprocessed_data/y_train.csv", index=False)
y_val.to_csv("preprocessed_data/y_val.csv", index=False)
y_test.to_csv("preprocessed_data/y_test.csv", index=False)

joblib.dump(scaler, "preprocessed_data/scaler.pkl")
joblib.dump(encoder, "preprocessed_data/encoder.pkl")
joblib.dump(num_imputer, "preprocessed_data/num_imputer.pkl")
joblib.dump(cat_imputer, "preprocessed_data/cat_imputer.pkl")

with open("preprocessed_data/numerical_cols.txt", "w") as f:
    f.write("\n".join(num_cols))

with open("preprocessed_data/categorical_cols.txt", "w") as f:
    f.write("\n".join(cat_cols))

print("\n==================== PREPROCESSING COMPLETE ====================\n")


==================== PREPROCESSING PIPELINE START ====================

Loaded raw data: (10000, 20)
After timestamp cleaning: (10000, 20)
Feature engineering complete (NO rolling features).
Removed leakage features: ['Historical Price', 'Total Revenue']
Feature matrix shape: (10000, 17)
Train: (7000, 17) | Val: (1500, 17) | Test: (1500, 17)
Numeric columns: ['Competitor Price', 'Inventory Level', 'Promotion Status', 'Purchase Quantity', 'Competitor Popularity', 'Seasonal Indicator', 'Price Elasticity', 'Customer Loyalty', 'competitor_price_difference', 'competitor_price_ratio', 'discount_percentage', 'price_change_rate']
Categorical columns: ['Product Name', 'Product Category', 'Customer Demographics', 'Customer Region', 'Market Segment']
Numeric imputation complete.
Categorical imputation complete.
Processed shapes: (7000, 107) (1500, 107) (1500, 107)

==================== PREPROCESSING COMPLETE ====================

